This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [1]:
figname = '../paper/fig_params'

In [2]:
%run 0_parameters.ipynb


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
Using filename= ../data/2019-03-20
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 50000, 'test_batch_size': 5000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 44, 'noise': 0.7, 'contrast': 0.8, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.5, 'dim1': 1000, 'dim2': 1000, 'lr': 0.0001, 'do_adam': True, 'bn1_bn_momentum': 0.0, 'bn2_bn_momentum': 0.0, 'momentum': 0.1, 'epochs': 50, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-20', 'seed': 2019, 'N_cv': 2}
Overwriting train.py
-rw-r--r--  1 laurentperrinet  wheel   397M Mar 19 15:38 /tmp/dataset_train__0.2_0.3_0.7_0.8_30_35_6_16_10_2_1.41_128_50000.pt
-rw-r--r--  1 laurentperr

In [3]:
%%writefile params.py
import torch
from main import init, MetaML

opts = dict(filename='../data/2019-03-16', verbose=0, log_interval=0)
#opts = dict(filename='debug', verbose=0, log_interval=0)

print(50*'-')
print(' parameter scan')
print(50*'-')

args = init(**opts)
args.do_compute = False

from where import Where as ML
from what import WhatNet

if False:
    print(50*'-')
    print('Default parameters')
    print(50*'-')

    ml = ML(args)
    ml.train(path=args.filename)
    # ml.main(path=args.filename)

if True:
    args = init(**opts)
    mml = MetaML(args)
    if torch.cuda.is_available():
        mml.scan('no_cuda', [True, False])

    args = init(**opts)
    mml = MetaML(args)
    mml.scan('bias_deconv', [True, False])
   

for base in [2, 8] if not args.filename == '../data/debug' else [2]:
    print(50*'-')
    print(' base=', base)
    print(50*'-')

    print(50*'-')
    print(' parameter scan : data')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args, base=base)
    for parameter in ['sf_0', 'B_sf', 'offset_std', 'noise', 'contrast']:
        mml.parameter_scan(parameter)


    print(50*'-')
    print(' parameter scan : network')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args)
    for parameter in ['dim1',
                      'bn1_bn_momentum',
                      'dim2',
                      'bn2_bn_momentum',
                      'p_dropout']:
        mml.parameter_scan(parameter)

    print(' parameter scan : learning ')
    args = init(**opts)
    mml = MetaML(args, base=base)
    print(50*'-')
    print('Using SGD')
    print(50*'-')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        mml.parameter_scan(parameter)
    print(50*'-')
    print('Using ADAM')
    print(50*'-')
    args = init(**opts)
    args.do_adam = True
    mml = MetaML(args, tag='adam')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        mml.parameter_scan(parameter)

    print(50*'-')
    print(' parameter scan : retina')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args)
    for parameter in ['N_theta',
                      'N_azimuth',
                      'N_eccentricity',
                      'rho']:
        mml.parameter_scan(parameter)
        

Overwriting params.py


In [ ]:
from where import Where as ML

In [ ]:
args.filename

'../data/2019-03-20'

In [ ]:
%run params.py

--------------------------------------------------
 parameter scan
--------------------------------------------------
scanning over bias_deconv = [True, False]
For parameter bias_deconv = True ,   currently locked with  ../data/_tmp_scanning/bias_deconv__True.npy_lock
For parameter bias_deconv = False ,  Accuracy=66.7% +/- 0.1%  in 6241.0 seconds
--------------------------------------------------
 base= 2
--------------------------------------------------
--------------------------------------------------
 parameter scan : data
--------------------------------------------------
scanning over sf_0 = [0.1        0.11892071 0.14142136 0.16817928 0.2        0.23784142
 0.28284271 0.33635857 0.4       ]
For parameter sf_0 = 0.100 ,  Accuracy=61.0% +/- 0.0%  in 8965.5 seconds
For parameter sf_0 = 0.119 ,  Accuracy=62.0% +/- 0.4%  in 9037.5 seconds
For parameter sf_0 = 0.141 ,  Accuracy=63.8% +/- 0.3%  in 9182.1 seconds
For parameter sf_0 = 0.168 ,  Accuracy=65.5% +/- 0.1%  in 9264.2 seconds


# summary


In [ ]:
%ls ../data/_tmp_scanning

# post-processing

In [ ]:
!ls  -l {figname}*